## Altair Kütüphanesiyle İett Ulaşım Verilerini Görselleştirme

Kütüphanelerimiz

In [1]:
import pandas as pd
import datetime
import altair as alt

Verimiz : https://www.istanbulkart.istanbul/ Kart Hareketlerim bölümünden Excel(xls formatı önerilir) olarak indirebilirsiniz. 

In [2]:

df=pd.read_excel('/work/HareketDokumu.xls')
df.tail()

,İşlem Numarası,İşlem Tarihi,İşlem Türü,İşlem Açıklaması,İşlem Tutarı (TL),İşlem Durumu,Kalan Hak
1029,1080011000,03.12.2018 17:10:33,Harcama Kontörlü - Ulaşım,MARMARAY-BC1,"0,55",Başarılı,"19,02"
1030,1080011000,03.12.2018 16:44:27,Harcama Kontörlü - Ulaşım,YENİKAPI - HAVALİMANI-M1,"1,25",Başarılı,"19,57"
1031,1080011000,03.12.2018 12:27:25,Harcama Kontörlü - Ulaşım,YENİKAPI - HAVALİMANI-M1,"0,50",Başarılı,"20,82"
1032,1080011000,03.12.2018 12:11:20,Harcama Kontörlü - Ulaşım,MARMARAY-BC1,"0,55",Başarılı,"21,32"
1033,1080011000,03.12.2018 11:12:20,Harcama Kontörlü - Ulaşım,ÜSKÜDAR-ÇAVUŞBAŞI/ KARANLIKDERE-15KÇ,"1,25",Başarılı,"21,87"


Data Manipülasyonları : 

In [3]:
df = df[df['İşlem Durumu']=='Başarılı'] # Başarısızları çıkaralım 
df['Hat'] = df["İşlem Açıklaması"].map( lambda  x: x.split('-')[-1]) # Ulaşım Hattını Elde Et
df['Tür'] = df["İşlem Türü"].map( lambda  x: x.split('- ')[-1]) # İşlem Türünü Elde Et
df["Tarih"]= pd.to_datetime(df["İşlem Tarihi"]) # tarih formatına çevir
df["Yil"] = pd.DatetimeIndex(df['Tarih']).year #Yıl
df["Ay"] = pd.DatetimeIndex(df["Tarih"]).month #Ay
df['Tutar'] = df['İşlem Tutarı (TL)'].str.replace(',', '.').astype('float') #Tutarı Floata çevirdik. 
df['Abonman'] = df['Tutar'].apply(lambda x: 0 if x > 0 else 1) # Tutar 0 dan küçükse Abonman Kullanım
data = df[['Tür','Hat','Tutar','Yil','Ay','Abonman']] # data mızı kullanacağımız alanlarla ayrı dataframe e aldık.
data  = data[data['Tür']=='Ulaşım'] # ulaşım olanları ayır.
data['Yil'] = data['Yil'].astype(str) # Yılı Stringe çevirdim kütüphane öyle istedi. 
data = data[data['Yil']!= '2018'] #2018 verileri eksik olduğu için çıkardım. 
data = data[data.groupby('Hat')['Hat'].transform('size')>15] # kullanımı 15 ten büyük olanları al
data['Abonman'].replace([1, 0], ['Abonman Kullanım','Normal Kullanım'], inplace = True) #Abonman kolonlarına yazı yazdım.
data #hazırız.

,Tür,Hat,Tutar,Yil,Ay,Abonman
0,Ulaşım,14,0.00,2020,12,Abonman Kullanım
1,Ulaşım,20,0.00,2020,12,Abonman Kullanım
2,Ulaşım,320,0.00,2020,12,Abonman Kullanım
3,Ulaşım,20,1.70,2020,12,Normal Kullanım
4,Ulaşım,13,1.70,2020,12,Normal Kullanım
...,...,...,...,...,...,...
974,Ulaşım,M1,0.50,2019,11,Normal Kullanım
977,Ulaşım,15KÇ,0.55,2019,10,Normal Kullanım
979,Ulaşım,M1,0.55,2019,10,Normal Kullanım
980,Ulaşım,M1,1.25,2019,10,Normal Kullanım


In [4]:
bars = alt.Chart(data).mark_bar(cornerRadiusTopLeft=3, # datamızı verdik.
    cornerRadiusTopRight=3).encode(
    y=alt.Y('count()', axis=alt.Axis(title='Kullanım Sayısı')), #y eksenine kullanım sayısı
    x=alt.X('Hat', sort='-y'), # x eksenine Hat bilgisi ve y eksenine göre sıralanması
   color=alt.Color('Yil'), # renklerin Yıllara göre ayarlanması.
   tooltip=[alt.Tooltip('count()', title='Kullanım Sayısı') , 'Hat','Yil'] #Üstüne gelince görülecek değerler.
).properties( 
    width=600,   
    height=300
)

texts = alt.Chart(data).mark_text(dy=-5,color='black').encode( # Üstteki text lerin yazılması. 
    y=alt.Y('count()', axis=alt.Axis(title='Kullanım Sayısı')), 
    x=alt.X('Hat', sort='-y'),
    text=alt.Text('count()', format='.0f')
)
bars + texts

alt.LayerChart(...)

##################################

Kütüphane : https://altair-viz.github.io/